### Section 1: Poloniex API Request

In [1]:
# Import necessary libraries
import os
import requests
import pandas as pd
from datetime import date

# Function to make API requests to Poloniex
def poloniex(endpoint: str, params: dict = {}):
    url = 'https://api.poloniex.com/'
    final_url = url + endpoint
    
    try:
        # Make the API request
        response = requests.get(final_url, params=params)

        # Check for a successful response
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Request failed with status code {response.status_code}")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")


### Section 2: Retrieve Ticker Data and Save to CSV


In [ ]:
# Get market tickers and create a DataFrame
market_tickers = poloniex(f'markets/ticker24h')
df = pd.DataFrame(market_tickers)

# Save the data to a CSV file named with the current date
today = date.today()
df.to_csv('Market-Data-Preprocessing\New-Api-Version\DataBase\Ticker24h/'+ str(today) + '.csv', index=False)

# Read the CSV file for further processing
df = pd.read_csv('Market-Data-Preprocessing\New-Api-Version\DataBase\Ticker24h/2023-10-27.csv')
df.head()

### Section 3: Data Cleaning and Adjustment

In [ ]:
# Remove test rows with specific prefixes
test_prefixes = ['POLOTEST1', 'POLOTEST2', 'POLOTEST3', 'POLOTEST4']
df = df[~df['symbol'].str.startswith(tuple(test_prefixes))]
df.reset_index(drop=True, inplace=True)

# Identify unique postfixes and adjust the data
unique_postfixes = df['symbol'].str.split('_').str[1].unique()

# Identify converting rates for currencies
# (Assumes that there is a base currency, represented by 'base_value')

### Section 4: Calculate Adjusted Volumes

In [ ]:
# Calculate adjusted volumes based on converting rates
df['adjusted_amount'] = df['amount'] * df['symbol'].str.split('_').str[1].map(converter_dict)

# Extract the coin names
df['coin'] = df['symbol'].str.split('_').str[0]

# Group by coin and calculate total adjusted volume
volumes_df = df.groupby('coin').agg({'adjusted_amount': 'sum'})
volumes_df.reset_index(drop=False, inplace=True)
volumes_df.rename(columns={'adjusted_amount': 'volume'}, inplace=True)

# Save the volumes data to a CSV file named with the current date
volumes_df.to_csv('Market-Data-Preprocessing\New-Api-Version\DataBase\Volumes/'+ str(today) + '.csv', index=False)

### Section 5: Aggregate Volumes Data

In [ ]:
# Aggregate volumes data from multiple CSV files
folder_path = 'Market-Data-Preprocessing\New-Api-Version\DataBase/Volumes'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
volumes_dicts = []

# Loop through CSV files and create dictionaries
for csv_file in csv_files:
    # Read each CSV file
    volumes_df = pd.read_csv(os.path.join(folder_path, csv_file))
    volumes_dict = volumes_df.set_index('coin')['volume'].to_dict()
    volumes_dicts.append(volumes_dict)

# Combine dictionaries to get final volumes
final_volumes_dict = {}
for d in volumes_dicts:
    for key, value in d.items():
        final_volumes_dict[key] = final_volumes_dict.get(key, 0) + value

# Create a DataFrame from the final volumes dictionary
final_volumes_df = pd.DataFrame.from_dict(final_volumes_dict, orient='index', columns=['volumes'])
final_volumes_df.reset_index(inplace=True)
final_volumes_df.columns = ['coin', 'volumes']

### Section 6: Display Top N Volumes

In [ ]:
# Define a function to get the top N volumes from a DataFrame
def top_n_volumes(dataframe, n=10, column_name='volumes'):
    sorted_dataframe = dataframe.sort_values(by=column_name, ascending=False)
    top_n_rows = sorted_dataframe.head(n)
    top_n_rows.reset_index(drop=True, inplace=True)
    return top_n_rows

# Display the top N volumes
print(top_n_volumes(final_volumes_df))

### Section 7: Get Coin Features

In [ ]:
# Define a function to get features for specific coins
def get_coins_features(coin_list, columns_list=['adjusted_amount']):
    grouped = df.groupby('coin')
    
    for coin in coin_list:
        selected_indices = grouped.get_group(coin).index
        print(f"Indices with {coin} as prefix:")
        for idx in selected_indices:
            print(df.loc[idx, 'symbol'], ": ", df.loc[idx, columns_list])
            print(' -------------------------- ')

# Example usage: Get features for coins with the prefix 'NFT'
get_coins_features(['NFT'], ['adjusted_amount', 'amount', 'high'])

### Section 8: Fetch Market Chart Data

In [ ]:
# Fetch market chart data for symbols in the top list
number_of_coins = 20
top_df = top_n_volumes(final_volumes_df, number_of_coins)
top_list = top_df['coin'].tolist()
symbols_list = [coin + '_' + base_value for coin in top_list]

# Fetch market chart data for each symbol in the list
for symbol in symbols_list:
    print(f'markets/{symbol}/candles')
    market_chart = poloniex(f'markets/{symbol}/candles', {'interval': 'HOUR_6'})
    chart_df = pd.DataFrame(market_chart)